In [26]:
# https://medium.com/@invest_gs/kaggle-housing-competition-learn-with-a-step-by-step-solution-bdca19cc8eed

# lightgbm example on house pricing

In [34]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df_train = pd.read_csv(r'data\house_prices\train.csv')
df_test = pd.read_csv(r'data\house_prices\test.csv')
samples_sumbition = pd.read_csv(r'data\house_prices\sample_submission.csv')

In [15]:
df_train.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [12]:
print(f'train shape: {df_train.shape}')
print(f'test shape: {df_test.shape}')

train shape: (1460, 81)
test shape: (1459, 80)


In [16]:
# combined train and test for easy exploration

# set SalePrice for test as -1 to separate train and test later
df_test['SalePrice'] = -1

# combine them
df_combined = pd.concat([df_train, df_test], axis=0)

In [20]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             2919 non-null   int64   
 1   MSSubClass     2919 non-null   category
 2   MSZoning       2915 non-null   object  
 3   LotFrontage    2433 non-null   float64 
 4   LotArea        2919 non-null   int64   
 5   Street         2919 non-null   object  
 6   Alley          198 non-null    object  
 7   LotShape       2919 non-null   object  
 8   LandContour    2919 non-null   object  
 9   Utilities      2917 non-null   object  
 10  LotConfig      2919 non-null   object  
 11  LandSlope      2919 non-null   object  
 12  Neighborhood   2919 non-null   object  
 13  Condition1     2919 non-null   object  
 14  Condition2     2919 non-null   object  
 15  BldgType       2919 non-null   object  
 16  HouseStyle     2919 non-null   object  
 17  OverallQual    2919 non-null   in

In [18]:
# Classify int variables into category if needed
df_combined["MSSubClass"] = df_combined["MSSubClass"].astype("category")
df_combined["MoSold"] = df_combined["MoSold"].astype("category")

In [21]:
# Categorical data impute with mode of neighborhood and MSSubClass or just mode of own column if missing
missing_vals = ["MSZoning", "Alley", "Utilities", 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',"Electrical",'KitchenQual','Functional','GarageType',"SaleType", 'GarageFinish','GarageQual','GarageCond','Exterior1st', 'Exterior2nd','FireplaceQu', "PoolQC", "Fence", "MiscFeature"]

for missing_val in missing_vals:
    try:
        df_combined[missing_val] = df_combined.groupby(['MSSubClass', "Neighborhood"])[missing_val].transform(lambda x: x.fillna(x.mode()[0]))
    except:
     df_combined[missing_val].fillna((df_combined[missing_val].mode()[0]), inplace=True)

In [22]:
# Add "Other" category as most elements are missing
df_combined["PoolQC"] = df_combined["PoolQC"].fillna("Other")

In [23]:
# Continuous data
missing_vals = ["LotFrontage", 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF1','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea',]
impute_vals = ["LotConfig" ,"Neighborhood",'BsmtFinType1', 'BsmtFinType2','BsmtQual', 'BsmtQual', 'BsmtQual','GarageType', 'GarageType']

for missing_val, impute_val in zip(missing_vals, impute_vals):
    df_combined[missing_val] = df_combined[missing_val].fillna(df_combined.groupby(impute_val)[missing_val].transform('mean'))

In [24]:
# Continuous impute data based on other continuous data
missing_vals = ['GarageYrBlt']
impute_vals = ['YearBuilt']

for missing_val, impute_val in zip(missing_vals, impute_vals):
    df_combined[missing_val] = df_combined[missing_val].fillna(df_combined[impute_val])

In [25]:
# Fill all leftovers with mean
for missing_val in df_combined.columns.values.tolist():

    if missing_val == "SalePrice":
        pass

    else:
        try:
            df_combined[missing_val] = df_combined[missing_val].fillna(df_combined[missing_val].mean())
        except:
            pass

# List of cols with missing values
print([col for col in df_combined.columns if df_combined[col].isnull().any()])

[]


In [32]:
# Add and change some variables, namely the "Year" ones as it would be better to have them as "Age"
year = datetime.date.today().year
df_combined["AgeSold"] = int(year) - df_combined["YrSold"].astype(int)
df_combined["AgeGarage"] = int(year) - df_combined["GarageYrBlt"].astype(int)
df_combined["AgeBuilt"] = int(year) - df_combined["YearBuilt"].astype(int)

In [33]:
# Add some features related to total area of the house
df_combined['TotalArea'] = df_combined['TotalBsmtSF'] + df_combined['1stFlrSF'] + df_combined['2ndFlrSF'] + df_combined['GrLivArea'] +df_combined['GarageArea']
df_combined['Bathrooms'] = df_combined['FullBath'] + df_combined['HalfBath']/2
df_combined['Year average'] = (df_combined['YearRemodAdd']+df_combined['YearBuilt'])/2

In [ ]:
# Check the sale price distribution by different types of variables
for element in ["MSSubClass", "MSZoning", "HouseStyle","CentralAir", "PoolQC", "SaleType"]:
    cat_plot = sns.catplot(y="SalePrice", x=element, kind="swarm", legend="full", data=df_combined, height=4.5, aspect=3/3,);
    cat_plot.set_xticklabels(rotation=90)